# Pickling

In [18]:
import numpy as np
import pandas as pd
from src.sequencing import create_sequence

def save_used_data(data, labels, used=None):
    """
    Concatenate data and labels, and optionally append to existing used data.
    
    Parameters:
    -----------
    data : ndarray
        Input data array of shape (n, m, 6)
    labels : ndarray
        Labels array of shape (n, m, k)
    used : ndarray, optional
        Previously used data array of shape (n, m, 6+l)
    
    Returns:
    --------
    ndarray
        Concatenated data array of shape (2n, m, 6+k) or (n, m, 6+k)
    """
    # Concatenate data and labels along the last axis
    combined_data = np.concatenate([data, labels], axis=2)
    
    # If no used data provided, return the combined data
    if used is None:
        return combined_data
    
    # Ensure labels dimension matches used data
    k = labels.shape[2]
    l = used.shape[2] - 6
    
    # Pad used data with zeros if needed
    if k != l:
        zero_pad = np.zeros((*used.shape[:2], k-l), dtype=used.dtype)
        used = np.concatenate([used, zero_pad], axis=2)
    
    # Concatenate used data vertically with combined data
    return np.concatenate([used, combined_data], axis=0)


<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\axelv\AppData\Local\Temp\ipykernel_12116\1988130870.py:1: SyntaxWarning: invalid escape sequence '\p'
  df = pd.read_csv('C:\projects\interactive-telemetry-for-design\data\GH010041_IMU.csv')


testing

In [ ]:
df = pd.read_csv('C:\projects\interactive-telemetry-for-design\data\GH010041_IMU.csv')
df = create_sequence(df, 0, 10)
data = df[:,:, 1:7]
labels = df[:,:,2:]
labels2 = df[:,:,1:]
labels3 = df[:,:,:]

t1= save_used_data(data, labels)
print(t1.shape)
t2 = save_used_data(data, labels, t1)
print(t2.shape)
t2a = save_used_data(data, labels2, t1)
print(t2a.shape)


In [ ]:
# load_dataframe('pickled_bytestream/imu_data.pkl')

load_multiple_dataframes('pickled_bytestream')